# IRGAN

Codigo:

https://github.com/geek-ai/irgan

paper: 

https://arxiv.org/pdf/1705.10513.pdf

Una GAN discreta (como SeqGan) pero que el generador es modelado como un politica de aprendizaje reforzado para seleccionar el documento candidato $d$ en el estado dado la query $q_n$, y una metrica de relevancia $r$ que es entrenado via una politica de gradientes

Quieren poner enfoque en el entrenamiento adversario, osea usan modelos simples e iguales para D y G

El módulo pickle implementa un algoritmo para convertir un objeto Python arbitrario en una serie de bytes. Este proceso también se llama serializar "el objeto". El flujo de bytes que representa el objeto se puede transmitir o almacenar y luego reconstruir para crear un nuevo objeto con las mismas características.

El módulo cPickle implementa el mismo algoritmo, en C en lugar de Python. Es muchas veces más rápido que la implementación de Python, pero no permite al usuario crear una subclase de Pickle. Si la subclasificación no es importante para su uso, probablemente desee utilizar cPickle.

In [ ]:
import _pickle as cPickle #guarda el modelo..
from IPython.display import Latex
import tensorflow as tf
import numpy as np
import multiprocessing
import pandas as pd 
import random
import pickle

cores = multiprocessing.cpu_count()
random.seed(0)


import matplotlib.pyplot as plt
import heapq
tf.compat.v1.disable_eager_execution()

## Discriminativo DNS

In [ ]:

class DIS():
    def __init__(self, itemNum, userNum, emb_dim,visual_matrix, visual_emb, lamda, param=None, initdelta=0.05, learning_rate=0.05,imageFeatureDim= 4096):
        self.itemNum = itemNum
        self.userNum = userNum
        self.emb_dim = emb_dim
        self.lamda = lamda  # regularization parameters
        self.param = param
        self.visual_I = visual_matrix #preentrenada no aprendible
        self.k2 = visual_emb
        self.initdelta = initdelta
        self.learning_rate = learning_rate
        self.imageFeatureDim= imageFeatureDim
        self.d_params = []

        with tf.compat.v1.compat.v1.variable_scope('discriminator'):
            if self.param is None: #Si no hay param -> inicializacion aleatoria
                self.user_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_bias = tf.compat.v1.Variable(tf.compat.v1.zeros([self.itemNum]))
                #####################################
                self.visual_U = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #E
                self.itemEmb_W = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.imageFeatureDim,self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #beta
                self.visual_item_bias = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum,self.imageFeatureDim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))

                ###################################################
            else: #lee los parametros del archivo
                self.user_embeddings = tf.compat.v1.Variable(self.param[0])
                self.item_embeddings = tf.compat.v1.Variable(self.param[1])
                self.item_bias = tf.compat.v1.Variable(self.param[2])
                #######################################
                self.visual_U = tf.compat.v1.Variable(self.param[3])
                self.itemEmb_W = tf.compat.v1.Variable(self.param[4])
                self.visual_item_bias = tf.compat.v1.Variable(self.param[5])
                ###########################
           
            self.d_params = [self.user_embeddings, self.item_embeddings, self.item_bias,  self.visual_U, self.itemEmb_W, self.visual_item_bias ]
        
        # placeholder definition
        self.u = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.pos = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.neg = tf.compat.v1.placeholder(tf.compat.v1.int32)

        self.u_embedding = tf.compat.v1.nn.embedding_lookup(self.user_embeddings, self.u)
        self.pos_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.pos)
        self.pos_bias = tf.compat.v1.gather(self.item_bias, self.pos)
        self.neg_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.neg)
        self.neg_bias = tf.compat.v1.gather(self.item_bias, self.neg)


        ##################################################################################

        self.visual_U_vector = tf.compat.v1.nn.embedding_lookup(self.visual_U, self.u)
        self.visual_I_matrix_pos = tf.compat.v1.nn.embedding_lookup(self.visual_I, self.pos) #[1,4096]
        self.visual_I_matrix_neg = tf.compat.v1.nn.embedding_lookup(self.visual_I, self.neg)
        self.visual_pos_bias = tf.compat.v1.gather(self.visual_item_bias, self.pos) #[1,4096]
        self.visual_neg_bias = tf.compat.v1.gather(self.visual_item_bias, self.neg)

       ##################

        self.Ef_pos= tf.matmul( self.visual_I_matrix_pos,self.itemEmb_W) #f_i x E= [1,4096]x[4096,k2]= [1,k2]
        self.Ef_neg= tf.matmul(self.visual_I_matrix_neg,self.itemEmb_W) 
        self.Ef= tf.compat.v1.matmul(self.itemEmb_W,self.visual_I,transpose_a=True,transpose_b=True ) #E^T x f^T =  [k2,4096]x[4096,#I] = [k2,#I]
        self.bf_pos=tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_pos, self.visual_pos_bias),1)#[1,4096]x[1,4096]
        self.bf_neg=  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_neg, self.visual_neg_bias),1)
          
        
        ##################
        
        self.pre_logits = tf.compat.v1.sigmoid(
            tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.pos_embedding - self.neg_embedding),1) # [1,k]x[1,k] multuply es element wise, reduce sum sum, osea producto punto.
             + self.pos_bias - self.neg_bias
             +  tf.compat.v1.reduce_sum( tf.compat.v1.multiply(self.visual_U_vector,self.Ef_pos- self.Ef_neg ) ,1 )  # [1,k2]x[1,k2]
             +self.bf_pos- self.bf_neg 
                          ) 
             
        
 
        self.reg= self.lamda * (
            tf.compat.v1.nn.l2_loss(self.u_embedding) +
            tf.compat.v1.nn.l2_loss(self.pos_embedding) +
            tf.compat.v1.nn.l2_loss(self.pos_bias) +
            tf.compat.v1.nn.l2_loss(self.neg_embedding) +
            tf.compat.v1.nn.l2_loss(self.neg_bias)+
            tf.compat.v1.nn.l2_loss(self.visual_U_vector) +
            tf.compat.v1.nn.l2_loss(self.itemEmb_W) +
            tf.compat.v1.nn.l2_loss(self.visual_neg_bias) +
            tf.compat.v1.nn.l2_loss(self.visual_pos_bias)
            )

        self.pre_loss = -tf.compat.v1.reduce_mean(tf.compat.v1.log(self.pre_logits)) + self.reg
        d_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)
        self.d_updates = d_opt.minimize(self.pre_loss, var_list=self.d_params)

        # for test stage, self.u: [batch_size]
        
                          # uxI= [batch_size,k]x[k,#I]=[batch_size,#I]                                                     # b_i= [1, #I]        # [batch_size, k2]x[k2, #I]  = [bact_size,#I]                                             #beta elementwise f = [#,4096]elementwise[#I,4096]= [1, #I]
        self.all_rating = tf.compat.v1.matmul(self.u_embedding, self.item_embeddings, transpose_a=False,transpose_b=True) + self.item_bias + tf.compat.v1.matmul(self.visual_U_vector, self.Ef, transpose_a=False, transpose_b=False) +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)                    

        #self.all_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias
        # for dns sample
        #esto da muy grande
                                                                        # [1,k]x[#I,k] = [1,#I]                                 #[1, #I]                                                      #[1,k2]x[#I,k2]=[1,#I]
        self.dns_rating = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)  

    def save_model(self, sess, filename):
        param = sess.run(self.d_params)
        cPickle.dump(param, open(filename, 'wb'))  #antes w

## utils.py

In [ ]:
import linecache
import numpy as np


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# Get batch data from training set
def get_batch_data(file, index, size):  # 1,5->1,2,3,4,5
    user = []
    item = []
    label = []
    for i in range(index, index + size):
        line = linecache.getline(file, i)
        line = line.strip()
        line = line.split()
        user.append(int(line[0]))
        user.append(int(line[0]))
        item.append(int(line[1]))
        item.append(int(line[2]))
        label.append(1.)
        label.append(0.)
    return user, item, label




## Generador

In [ ]:
class GEN():
    def __init__(self, itemNum, userNum, emb_dim, visual_matrix, visual_emb, lamda, param=None, initdelta=0.05, learning_rate=0.05,imageFeatureDim= 4096):
        self.itemNum = itemNum
        self.userNum = userNum
        self.emb_dim = emb_dim
        self.lamda = lamda  # regularization parameters
        self.param = param
        self.initdelta = initdelta
        self.learning_rate = learning_rate
        self.g_params = []
        self.visual_I = visual_matrix #preentrenada no aprendible
        self.k2 = visual_emb
        self.imageFeatureDim= imageFeatureDim

        with tf.compat.v1.variable_scope('generator'):
            if self.param == None: #inicialización de representaciones
                self.user_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_bias = tf.compat.v1.Variable(tf.compat.v1.zeros([self.itemNum]))
                #####################################
                self.visual_U = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #E
                self.itemEmb_W = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.imageFeatureDim,self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #beta
                self.visual_item_bias = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum,self.imageFeatureDim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))

                ###################################################


            else:
                self.user_embeddings = tf.compat.v1.Variable(self.param[0])
                self.item_embeddings = tf.compat.v1.Variable(self.param[1])
                self.item_bias = tf.compat.v1.Variable(param[2])
                                #######################################
                self.visual_U = tf.compat.v1.Variable(self.param[3])
                self.itemEmb_W = tf.compat.v1.Variable(self.param[4])
                self.visual_item_bias = tf.compat.v1.Variable(self.param[5])
                ###########################

            self.g_params = [self.user_embeddings, self.item_embeddings, self.item_bias,self.visual_U, self.itemEmb_W, self.visual_item_bias ]

        self.u = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.i = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.reward = tf.compat.v1.placeholder(tf.compat.v1.float32)

        self.u_embedding = tf.compat.v1.nn.embedding_lookup(self.user_embeddings, self.u)
        self.i_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.i)
        self.i_bias = tf.compat.v1.gather(self.item_bias, self.i)


        ##################################################################################

        self.visual_U_vector = tf.compat.v1.nn.embedding_lookup(self.visual_U, self.u)
        self.visual_I_matrix_i= tf.compat.v1.nn.embedding_lookup(self.visual_I, self.i) #[1,4096]
        self.visual_bias_i = tf.compat.v1.gather(self.visual_item_bias, self.i) #[1,4096]

       ##################

        self.Ef_i= tf.matmul( self.visual_I_matrix_i,self.itemEmb_W) #f_i x E= [1,4096]x[4096,k2]= [1,k2]
        self.Ef= tf.compat.v1.matmul(self.itemEmb_W,self.visual_I,transpose_a=True,transpose_b=True ) #E^T x f^T =  [k2,4096]x[4096,#I] = [k2,#I]
        self.bf_i=tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_i, self.visual_bias_i),1)#[1,4096]x[1,4096]
          
        
        ##################
                                                                                                           
                                                                        #[1,k]x[#I,k]                                   #[1,#I]                                # [1,k2]x[1,k2]
        self.all_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)  

        self.i_prob = tf.compat.v1.gather(
            tf.compat.v1.reshape(tf.compat.v1.nn.softmax(tf.compat.v1.reshape(self.all_logits, [1, -1])), [-1]),
            self.i)

        self.reg= self.lamda * (tf.compat.v1.nn.l2_loss(self.u_embedding) + tf.compat.v1.nn.l2_loss(self.i_embedding) + tf.compat.v1.nn.l2_loss(self.i_bias)+
                                 tf.compat.v1.nn.l2_loss(self.visual_U_vector) +tf.compat.v1.nn.l2_loss(self.itemEmb_W) +tf.compat.v1.nn.l2_loss(self.visual_bias_i) )
        
        self.gan_loss = -tf.compat.v1.reduce_mean(tf.compat.v1.log(self.i_prob) * self.reward) + self.reg

        g_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)

        self.gan_updates = g_opt.minimize(self.gan_loss, var_list=self.g_params)

        # for test stage, self.u: [batch_size]
        self.all_rating = tf.compat.v1.matmul(self.u_embedding, self.item_embeddings, transpose_a=False,
                                    transpose_b=True) + self.item_bias + tf.compat.v1.matmul(self.visual_U_vector, self.Ef, transpose_a=False, transpose_b=False) +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1) 

    def save_model(self, sess, filename):
        param = sess.run(self.g_params)
        cPickle.dump(param, open(filename, 'wb')) #mismo cambio

## Discriminador

$f_{\phi} (q,d)= s_{\phi} (q,d)= s(u,i)= b_i + v_u^Tv_i$

Máquina de factorización sencilla

Su labor es distinguir (clasificar) si el documento $d$ es relevante (real, etiqueta=1, ejemplo positivo) para la consulta $q$. Estimando la probabilidad:

$D(d|q)= \sigma (f_{\phi} (q,d))= \dfrac{exp(f_{\phi} (q,d))}{1+ exp(f_{\phi} (q,d))} $

Su objetivo entonces es maximiza la log verosimititud de distrinción entre documentos verdaderos y generados:

$\phi^{*}=\arg \max _{\phi} \sum_{n=1}^{N}\left(\mathbb{E}_{d \sim p_{\text {true }}\left(d \mid q_{n}, r\right)}\left[\log \left(\sigma\left(f_{\phi}\left(d, q_{n}\right)\right)\right]+\right.\right.$
$\left.\mathbb{E}_{d \sim p_{\theta^{*}}\left(d \mid q_{n}, r\right)}\left[\log \left(1-\sigma\left(f_{\phi}\left(d, q_{n}\right)\right)\right)\right]\right)$

Modelo generativo actual fijo

In [ ]:
class DIS():
    def __init__(self, itemNum, userNum, emb_dim, visual_matrix, visual_emb,lamda, param=None, initdelta=0.05, learning_rate=0.05,imageFeatureDim=4096):
        self.itemNum = itemNum
        self.userNum = userNum
        self.emb_dim = emb_dim
        self.lamda = lamda  # regularization parameters
        self.param = param
        self.initdelta = initdelta
        self.learning_rate = learning_rate
        self.d_params = []
        self.visual_I = visual_matrix #preentrenada no aprendible
        self.k2 = visual_emb
        self.imageFeatureDim= imageFeatureDim

        with tf.compat.v1.variable_scope('discriminator'):
          #inicializacion
            if self.param == None:
                self.user_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_embeddings = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum, self.emb_dim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                self.item_bias = tf.compat.v1.Variable(tf.compat.v1.zeros([self.itemNum]))
                #####################################
                self.visual_U = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.userNum, self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #E
                self.itemEmb_W = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.imageFeatureDim,self.k2], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))
                #beta
                self.visual_item_bias = tf.compat.v1.Variable(
                    tf.compat.v1.random_uniform([self.itemNum,self.imageFeatureDim], minval=-self.initdelta, maxval=self.initdelta,
                                      dtype=tf.compat.v1.float32))

                ###################################################


            else:
                self.user_embeddings = tf.compat.v1.Variable(self.param[0])
                self.item_embeddings = tf.compat.v1.Variable(self.param[1])
                self.item_bias = tf.compat.v1.Variable(self.param[2])
                #######################################
                self.visual_U = tf.compat.v1.Variable(self.param[3])
                self.itemEmb_W = tf.compat.v1.Variable(self.param[4])
                self.visual_item_bias = tf.compat.v1.Variable(self.param[5])
                ###########################

        self.d_params = [self.user_embeddings, self.item_embeddings, self.item_bias,self.visual_U, self.itemEmb_W, self.visual_item_bias ]

        # placeholder definition
        self.u = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.i = tf.compat.v1.placeholder(tf.compat.v1.int32)
        self.label = tf.compat.v1.placeholder(tf.compat.v1.float32)
        #indexaciones
        #V_u
        self.u_embedding = tf.compat.v1.nn.embedding_lookup(self.user_embeddings, self.u)
        #V_i
        self.i_embedding = tf.compat.v1.nn.embedding_lookup(self.item_embeddings, self.i)
        #b_i
        self.i_bias = tf.compat.v1.gather(self.item_bias, self.i)


        ##################################################################################

        self.visual_U_vector = tf.compat.v1.nn.embedding_lookup(self.visual_U, self.u)
        self.visual_I_matrix_i= tf.compat.v1.nn.embedding_lookup(self.visual_I, self.i) #[1,4096]
        self.visual_bias_i = tf.compat.v1.gather(self.visual_item_bias, self.i) #[1,4096]

       ##################
        self.Ef_i= tf.matmul( self.visual_I_matrix_i,self.itemEmb_W) #f_i x E= [1,4096]x[4096,k2]= [1,k2]
        self.Ef= tf.compat.v1.matmul(self.itemEmb_W,self.visual_I,transpose_a=True,transpose_b=True ) #E^T x f^T =  [k2,4096]x[4096,#I] = [k2,#I]
        self.bf_i=tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_I_matrix_i, self.visual_bias_i),1)#[1,4096]x[1,4096]
          
        
        ##################

        #b_i+vu*v_i
        self.pre_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.i_embedding), 1) + self.i_bias + tf.compat.v1.reduce_sum( tf.compat.v1.multiply(self.visual_U_vector,self.Ef_i ) ,1 ) + self.bf_i
        #sigmoide cross entropy con regularización l2 
        self.reg= self.lamda * (
            tf.compat.v1.nn.l2_loss(self.u_embedding) + tf.compat.v1.nn.l2_loss(self.i_embedding) + tf.compat.v1.nn.l2_loss(self.i_bias)+
                                 tf.compat.v1.nn.l2_loss(self.visual_U_vector) +tf.compat.v1.nn.l2_loss(self.itemEmb_W) +tf.compat.v1.nn.l2_loss(self.visual_bias_i) )
        
        self.pre_loss = tf.compat.v1.nn.sigmoid_cross_entropy_with_logits(labels=self.label,
                                                                logits=self.pre_logits) + self.reg
        #SGD
        d_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)
        self.d_updates = d_opt.minimize(self.pre_loss, var_list=self.d_params)
        #Recompensa: 
        #b_i+vu*v_i
        self.reward_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.i_embedding),
                                           1) + self.i_bias +  tf.compat.v1.reduce_sum( tf.compat.v1.multiply(self.visual_U_vector,self.Ef_i ) ,1 ) + self.bf_i
        #2sig(f)-1
        self.reward = 2 * (tf.compat.v1.sigmoid(self.reward_logits) - 0.5)

        # for test stage, self.u: [batch_size]
        self.all_rating = tf.compat.v1.matmul(self.u_embedding, self.item_embeddings, transpose_a=False,
                                    transpose_b=True) + self.item_bias + tf.compat.v1.matmul(self.visual_U_vector, self.Ef, transpose_a=False, transpose_b=False) +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1) 

        #1 usuario todos los items.
        
        self.all_logits = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias +  tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1)  
        
        #self.NLL = -tf.compat.v1.reduce_mean(tf.compat.v1.log(
            #tf.compat.v1.gather(tf.compat.v1.reshape(tf.compat.v1.nn.softmax(tf.compat.v1.reshape(self.all_logits, [1, -1])), [-1]), self.i)))
        # for dns sample
         
        self.dns_rating = tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.u_embedding, self.item_embeddings), 1) + self.item_bias + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_U_vector, tf.transpose(self.Ef) ), 1) + tf.compat.v1.reduce_sum(tf.compat.v1.multiply(self.visual_item_bias , self.visual_I),1) 

    def save_model(self, sess, filename):
        param = sess.run(self.d_params)
        cPickle.dump(param, open(filename, 'w'))

# Adaptación de IRGAN

## Lectura Amazon 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#workdir = '/content/drive/MyDrive/dataset_ml/amazon_men_actualizado/' 
workdir = '/content/drive/MyDrive/dataset_ml/women/' 

usuarios_train = np.load(workdir+'dic_train_women.npy',allow_pickle='TRUE')
usuarios_train=list(usuarios_train.reshape(-1,1))[0][0]
usuarios_test = np.load(workdir+'dic_test_women.npy',allow_pickle='TRUE')
usuarios_test=list(usuarios_test.reshape(-1,1))[0][0]
DIS_TRAIN_FILE =workdir+ "dis-train.txt"
DIS_MODEL_FILE =   workdir+"model_dns_IRGANvisual.pkl" #rutaaaa

Mounted at /content/drive


In [ ]:
dic_feautures = np.load(workdir+'features_women_effnet.npy',allow_pickle='TRUE')
dic_feautures=list(dic_feautures.reshape(-1,1))[0][0]

In [ ]:
#########################################################################################
# Hyper-parameters
#########################################################################################
EMB_DIM = 10
K2=500
DNS_K = 5

In [ ]:
user_pos_test = {}
user_pos_train= {}
c_t=0
all_items= []


for u in usuarios_train.keys():
  for item in usuarios_train[u]:
    if item[b"productid"] not in all_items:
      all_items.append(item[b"productid"])
    
    if u not in user_pos_train: #nuevo
      user_pos_train[u] = [item[b"productid"]]
    else: #sino agrego
      user_pos_train[u].append(item[b"productid"])
all_user= list(user_pos_train.keys())

for u in usuarios_train.keys():    
  item=usuarios_test[u][0]
  if item[b"productid"] not in all_items:
    c_t+=1
  else:
    user_pos_test[u] = [item[b"productid"]]

ITEM_NUM=len(all_items)
USER_NUM= len(usuarios_train.keys())

In [ ]:
print("% de items en test que no estan en train: ",c_t/len(all_items)*100)
print("n item: ", ITEM_NUM, "n usuarios: ", USER_NUM)

% de items en test que no estan en train:  2.7743271221532093
n item:  7245 n usuarios:  3158


In [ ]:
c=0
for i in all_items:
  vector= dic_feautures[i]
  if c==0:
    visual_matrix= vector
    c=1
  else:
    visual_matrix= np.concatenate((visual_matrix, vector),axis=0)

In [ ]:
print(visual_matrix.shape)

(9106, 1536)


In [ ]:
np.max(visual_matrix)

21.523172

In [ ]:
visual_matrix= visual_matrix/np.max(visual_matrix)

## Inicialiazacion DNS

In [ ]:
def generate_dns(sess, model_, filename): #dy
    """"
    crea tripletas u,i,j con los id originales
    """
    data = []
    for u in list(user_pos_train.keys()): #para cada usuario
        pos = user_pos_train[u] #lo que consumio
        all_rating = sess.run(model_.dns_rating, {model_.u: all_user.index(u)}) #evalua el modelo
        all_rating = np.array(all_rating)
        neg = []
        candidates = list(set(all_items) - set(pos) )#los que no ha consumido

        for _ in range(len(pos)): #para cada item
            choice = np.random.choice(candidates, DNS_K) #se escogen DN_K aleatoriamente de los candidatos
            choice= [list(all_items).index(i) for i in choice]
            choice_score = all_rating[choice] # sus rating
            neg.append(choice[np.argmax(choice_score)] ) #se escoge el items mas cercanos al gusto de esta muestra aleatoria 
            #top_10_ind=random.sample(heapq.nlargest(10, range(len(choice_score)), choice_score.take),1)
            #neg.append(choice[top_10_ind[0]])
        for i in range(len(pos)): #para cada item
            data.append( str(all_user.index(u)) + '\t' + str(list(all_items).index(pos[i]) ) + '\t' + str(neg[i]) )
            
    with open(filename, 'w')as fout:
        fout.write('\n'.join(data))



def AUC(rating, test_users,dict_tov):
  """
  para cada usuario cuenta cuantos items de una muestra aleatoria tiene rating menor a 
  al rating del item consumido en test
  """
  ans=0
  cc=0
  for user in test_users:
      user_ind= test_users.index(user)
      user_= all_user[user]
      if dict_tov[user_][0] in all_items:
          item_test= list(all_items).index(dict_tov[user_][0])
          cc+=1
          items_train= [ list(all_items).index(i) for i in user_pos_train[user_]]  
          no_considerar= set(items_train+ [item_test]) 
          count=0
          tmpans=0 
          for j in random.sample(range(ITEM_NUM),int(50*(len(no_considerar)-1))): #sample
              if j in no_considerar: continue
              if rating[user_ind,item_test]>rating[user_ind,j]: tmpans+=1
              count+=1

          tmpans/=float(count)
          ans+=tmpans
  
  ans/=float(cc)
  return ans



def simple_test(sess, model, dict_tov): 
  #Calcula AUC para todos los usuarios comparada con el elemento que quedo en el test set
  #dict_tov: test o validacion
 test_users=[ all_user.index(i) for i in dict_tov.keys()]
 user_batch_rating = sess.run(model.all_rating, {model.u: test_users}) 
 batch_result= AUC(user_batch_rating,test_users, dict_tov)
    
 return batch_result

In [ ]:
tf.compat.v1.reset_default_graph()
param = None
discriminator = DIS(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2, lamda=0.001, param=param, initdelta=0.1, learning_rate=0.1, imageFeatureDim=1536) #se llama al discriminador DNS


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())

generate_dns(sess, discriminator, DIS_TRAIN_FILE)

In [ ]:
dis_log = open(workdir+'dis_log_dns_IRGANvisual.txt', 'w')
best_AUC=simple_test(sess, discriminator, user_pos_train)
#print(sess.run(discriminator.pre_loss,feed_dict={discriminator.u: [0], discriminator.pos: [1],discriminator.neg: [2]}))
#print("auc discriminador inicializado random", best_AUC)
losses=[]
aucs=[]
for epoch in range(3): #80 antes
  loss_d_=0
  c=0
  generate_dns(sess, discriminator, DIS_TRAIN_FILE)  # dynamic negative sample tarda 20 seg
  with open(DIS_TRAIN_FILE)as fin:

          for line in fin:
              line = line.split()
              c+=1
              u = int(line[0])
              i = int(line[1])
              j = int(line[2])
              #se actualiza el discriminador para la nueva tripleta
              _ ,loss= sess.run([discriminator.d_updates, discriminator.pre_loss ],
                             feed_dict={discriminator.u: [u], discriminator.pos: [i],
                                        discriminator.neg: [j]})
            
            
              #print(line)
              #print(loss, "  :loss")
              #print(sess.run([discriminator.reg, discriminator.pre_logits , discriminator.pre_loss],feed_dict={discriminator.u: [u], discriminator.pos: [i],discriminator.neg: [j]}), "  :reg , prelogits")
              loss_d_+=loss
                
  losses.append(loss_d_/c)
  AUC_actual = simple_test(sess, discriminator,user_pos_train) #evalua
  print ("epoch ", epoch, "dis: ", AUC_actual, "loss: ",loss_d_/c )
  aucs.append(AUC_actual)
    
  if AUC_actual> best_AUC:
          print("mejore")
          best_AUC = AUC_actual
          discriminator.save_model(sess, DIS_MODEL_FILE)
  
  print ("best AUC: ", best_AUC)

  buf = '\t'.join([str(AUC_actual)])
  dis_log.write(str(epoch) + '\t' + buf + '\n')
  dis_log.flush()

dis_log.close()

epoch  0 dis:  0.8900728626147765 loss:  0.9419832744527792
mejore
best AUC:  0.8900728626147765
epoch  1 dis:  0.9104114638250504 loss:  0.8962275549279018
mejore
best AUC:  0.9104114638250504


In [ ]:
losses

In [ ]:
aucs

## minmax

la regularizacion es crucial

In [ ]:
BATCH_SIZE = 1000 #16
import pickle
print ("load model...")


with open( workdir+ "model_dns_IRGANvisual.pkl", 'rb') as f: 
  param = pickle.load(f,encoding="bytes")

generator = GEN(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2,lamda=0.001, param=param, initdelta=0.1,
                    learning_rate=0.01, imageFeatureDim=1536 )

discriminator = DIS(ITEM_NUM, USER_NUM, EMB_DIM,visual_matrix, K2, lamda=0.001, param=None, initdelta=0.1,
                        learning_rate=0.01, imageFeatureDim=1536) #segundo discriminador (el que no es DNS)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())
g1= simple_test(sess, generator,user_pos_train)
d1=simple_test(sess, discriminator,user_pos_train)
print ("gen ", g1)
print ("dis ", d1)

load model...
gen  0.9116949460266968
dis  0.49850717191236965


load model...
gen  0.9116949460266968
dis  0.49850717191236965

In [ ]:

def generate_for_d(sess, model, filename):
    data = []
    for u in user_pos_train: #para cada usario
        pos = user_pos_train[u] #sus items 
        rating = sess.run(model.all_rating, {model.u: [all_user.index(u)]}) #completar la matriz 
        rating = np.array(rating[0]) #/ 0.2  # Temperature #infla los rating #plotear que pasa con la sigmoide cuando ponemos temperatura
        exp_rating = np.exp(rating) 
        prob = exp_rating / np.sum(exp_rating)
        #print(u)
        neg = np.random.choice(list(all_items), size=len(pos), p=prob) #muestreo negativo 
        for i in range(len(pos)):
            data.append( str(all_user.index(u)) + '\t' + str(list(all_items).index(pos[i]) ) + '\t' + str(list(all_items).index(neg[i])) ) 

    with open(filename, 'w')as fout:
        fout.write('\n'.join(data))



In [ ]:
# minimax training
dis_log = open('dis_log.txt', 'w')
gen_log = open('gen_log.txt', 'w')
loss_d=[]
auc_epocs_g=[]
auc_epocs_d=[]
loss_g=[]
for epoch in range(15):#15
    print("inicia minmax")
    if epoch >= 0:
        #entrena D 
        for d_epoch in range(10):##100 #para el generador fijo, entrena al discriminador
           # print("empieza a entrenarse el discriminador")
            loss_d_=0
            if d_epoch % 5 == 0:
                generate_for_d(sess, generator, DIS_TRAIN_FILE) #crea un archivo con las tripletas
                #print("pase")
                train_size = file_len(DIS_TRAIN_FILE)
            index = 1
            c=0
            while index < train_size-BATCH_SIZE: #hacer minibatch con etiquetado si es ejemplo positivo o negativo
                if index + BATCH_SIZE <= train_size + 1:
                    input_user, input_item, input_label = get_batch_data(DIS_TRAIN_FILE, index, BATCH_SIZE)
                else:
                    input_user, input_item, input_label = get_batch_data(DIS_TRAIN_FILE, index,
                                                                            train_size - index + 1)
                index += BATCH_SIZE
               # print(index)
                _, loss_dis = sess.run([discriminator.d_updates, discriminator.pre_loss],
                                 feed_dict={discriminator.u: input_user, discriminator.i: input_item, discriminator.label: input_label}) #actualizaciones de los embeddings y bias tales que clasifiquen bien
                loss_d_+=loss_dis
                c+=1
            
            loss_d.append(loss_d_/c)
        
            auc_actual = simple_test(sess, discriminator, user_pos_train)
            auc_epocs_d.append(auc_actual)
            #print ("epoch ", epoch, "dis: ", auc_actual)
            if auc_actual > best_AUC:
                print ('mejore: ', auc_actual)
                best_AUC = auc_actual

        for g_epoch in range(5):  # 50 #para el discriminador fijo (con pesos actualizados), se entrena el generador
            #print("empieza a entrenarse el generador")
            c=0
            loss_g_=0
            for u in user_pos_train: #por cada usuario
                sample_lambda = 0.2 #temperatura
                pos = [list(all_items).index(i) for i in user_pos_train[u]] #peliculas relevantes
                rating = sess.run(generator.all_logits, {generator.u: all_user.index(u)}) #calcula los ratings para ese usuario
                exp_rating = np.exp(rating)
                prob = exp_rating / np.sum(exp_rating)  # prob is generator distribution p_\theta
                pn = (1 - sample_lambda) * prob
                pn[pos] += sample_lambda * 1.0 / len(pos) #pedirle las prob a lo consumido + 0.2/la cantidad de items consumidos
                # Now, pn is the Pn in importance sampling, prob is generator distribution p_\theta
                sample = np.random.choice(np.arange(ITEM_NUM), 2 * len(pos), p=pn) #elige el doble de items c/r a lo consumido
                
                #               # Get reward and adapt it with importance sampling                #######
                reward = sess.run(discriminator.reward, {discriminator.u: all_user.index(u), discriminator.i: sample})
                reward = reward * prob[sample] / pn[sample]
                ######             # Update G              ####################
                _, loss_gen = sess.run([generator.gan_updates, generator.gan_loss],{generator.u: all_user.index(u), generator.i: sample, generator.reward: reward}) #actualiza pesos
                c+=1
                loss_g_+=loss_gen
                
            loss_g.append(loss_g_/c)
            auc_actual = simple_test(sess, generator, user_pos_train)
            auc_epocs_g.append(auc_actual)
            print ("epoch ", epoch, "gen: ", auc_actual)
            buf = '\t'.join([str(auc_actual) ])
            gen_log.write(str(epoch) + '\t' + buf + '\n')
            gen_log.flush()
            if auc_actual > best_AUC:
                print ('mejore: ', auc_actual)
                best_AUC = auc_actual
                generator.save_model(sess, "gan_generator.pkl") #se guarda el mejor modelo, la mejor factorización matricial.

gen_log.close()
dis_log.close()



inicia minmax
epoch  0 gen:  0.8959414851080767
mejore:  0.8959414851080767
epoch  0 gen:  0.8995279548939695
mejore:  0.8995279548939695
epoch  0 gen:  0.8976934032393626
epoch  0 gen:  0.8969843891797786
epoch  0 gen:  0.8946711502769191
inicia minmax
epoch  1 gen:  0.8904483319985119
epoch  1 gen:  0.8875710015029633
epoch  1 gen:  0.8853620669252406
epoch  1 gen:  0.8822660399973065
epoch  1 gen:  0.8782526398068847
inicia minmax
epoch  2 gen:  0.8752067377718498
epoch  2 gen:  0.8697559470309231
epoch  2 gen:  0.8666775232741865
epoch  2 gen:  0.8628582728018616
epoch  2 gen:  0.8590917068029897
inicia minmax
epoch  3 gen:  0.8533357582707628
epoch  3 gen:  0.850087628490991
epoch  3 gen:  0.8457106726255152
epoch  3 gen:  0.8410731689149065
epoch  3 gen:  0.8369502689989318
inicia minmax
epoch  4 gen:  0.8326232939310749
epoch  4 gen:  0.8266149859475033
epoch  4 gen:  0.8229670725544892
epoch  4 gen:  0.8176627799442207
epoch  4 gen:  0.8128138728168106
inicia minmax
epoch  5 ge

In [ ]:
print(simple_test(sess, discriminator,user_pos_train))
print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, discriminator,user_pos_test))
print(simple_test(sess, generator,user_pos_test))


textfile = open(workdir+"IRGAN_visual.txt", "w")
textfile.write(str(loss_d) + "\n")
textfile.write(str(auc_epocs_g) + "\n")
textfile.write(str(auc_epocs_d) + "\n")
textfile.write(str(loss_g) + "\n")
textfile.close()

NameError: ignored

In [ ]:
with open( workdir+ "gan_generator.pkl", 'rb') as f: 
  param = pickle.load(f,encoding="bytes")

generator = GEN(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2,lamda=0.001, param=param, initdelta=0.1,
                    learning_rate=0.01, imageFeatureDim=1536 )
textfile = open(workdir+"IRGAN_visual.txt", "w")
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())


print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, generator,user_pos_test))

0.8918364826814386
0.7962727415096519


# boots

## Inicialiazacion DNS

In [ ]:
DIS_TRAIN_FILE =workdir+ "dis-train_boots.txt"
DIS_MODEL_FILE =   workdir+"model_dns_IRGANvisual_boots.pkl" #rutaaaa

In [ ]:



def AUC(rating, test_users,dict_tov):
  """
  para cada usuario cuenta cuantos items de una muestra aleatoria tiene rating menor a 
  al rating del item consumido en test
  """
  ans=0
  cc=0
  for user in test_users:
      user_ind= test_users.index(user)
      user_= all_user[user]
      if dict_tov[user_][0] in all_items:
          item_test= list(all_items).index(dict_tov[user_][0])
          cc+=1
          items_train= [ list(all_items).index(i) for i in user_pos_train[user_]]  
          no_considerar= set(items_train+ [item_test]) 
          count=0
          tmpans=0 
          for j in random.sample(range(ITEM_NUM),int(50*(len(no_considerar)-1))): #sample
              if j in no_considerar: continue
              if rating[user_ind,item_test]>rating[user_ind,j]: tmpans+=1
              count+=1

          tmpans/=float(count)
          ans+=tmpans
  
  ans/=float(cc)
  return ans



def simple_test(sess, model, dict_tov): 
  #Calcula AUC para todos los usuarios comparada con el elemento que quedo en el test set
  #dict_tov: test o validacion
 test_users=[ all_user.index(i) for i in dict_tov.keys()]
 user_batch_rating = sess.run(model.all_rating, {model.u: test_users}) 
 batch_result= AUC(user_batch_rating,test_users, dict_tov)
    
 return batch_result

In [ ]:
tf.compat.v1.reset_default_graph()
param = None
discriminator = DIS(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2, lamda=0.001, param=param, initdelta=0.1, learning_rate=0.1, imageFeatureDim=1536) #se llama al discriminador DNS


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())

#generate_dns(sess, discriminator, DIS_TRAIN_FILE)

no sirve considerar todos ni tomar el top algo sobre una muestra mas grande

In [ ]:
%%time
dis_log = open(workdir+'dis_log_dns_IRGANvisual.txt', 'w')
best_AUC=simple_test(sess, discriminator, user_pos_train)
#print(sess.run(discriminator.pre_loss,feed_dict={discriminator.u: [0], discriminator.pos: [1],discriminator.neg: [2]}))
#print("auc discriminador inicializado random", best_AUC)
losses=[]
DNS_K= 5
aucs=[]
for epoch in range(int(13000)): 
        u= np.random.choice(list(user_pos_train.keys()),1)[0]
        i = list(all_items).index(np.random.choice(user_pos_train[u],1)[0] ) #lo que consumio
        pos = user_pos_train[u]
        all_rating = sess.run(discriminator.dns_rating, {discriminator.u: all_user.index(u)}) #evalua el modelo
        all_rating = np.array(all_rating)
        neg = []
        candidates = list(set(all_items) - set(pos) )#los que no ha consumido
        
        choice = np.random.choice(candidates, DNS_K) #se escogen DN_K aleatoriamente de los candidatos
        choice= [list(all_items).index(i) for i in choice]
        j= np.random.choice(choice,1)[0]
        u= all_user.index(u)

              #se actualiza el discriminador para la nueva tripleta
        _ ,loss= sess.run([discriminator.d_updates, discriminator.pre_loss ],
                             feed_dict={discriminator.u: [u], discriminator.pos: [i],
                                        discriminator.neg: [j]})
                
        if epoch%1000==0:
          losses.append(loss)
          AUC_actual = simple_test(sess, discriminator,user_pos_train) #evalua
          print ("epoch ", epoch, "dis: ", AUC_actual, "loss: ",loss )
          aucs.append(AUC_actual)
            
          if AUC_actual> best_AUC:
                  print("mejore")
                  best_AUC = AUC_actual
                  discriminator.save_model(sess, DIS_MODEL_FILE)
                  print ("best AUC: ", best_AUC)
                  
        buf = '\t'.join([str(AUC_actual)])
        dis_log.write(str(epoch) + '\t' + buf + '\n')
        dis_log.flush()

dis_log.close()

epoch  0 dis:  0.5012954302627196 loss:  2.0601177
mejore
best AUC:  0.5012954302627196
epoch  1000 dis:  0.6149333506607484 loss:  1.1278698
mejore
best AUC:  0.6149333506607484
epoch  2000 dis:  0.657589176761334 loss:  1.404794
mejore
best AUC:  0.657589176761334
epoch  3000 dis:  0.6896867464457391 loss:  1.199521
mejore
best AUC:  0.6896867464457391
epoch  4000 dis:  0.7054822704227821 loss:  1.2387245
mejore
best AUC:  0.7054822704227821
epoch  5000 dis:  0.722411302996846 loss:  4.0489035
mejore
best AUC:  0.722411302996846
epoch  6000 dis:  0.742912918926421 loss:  0.6096073
mejore
best AUC:  0.742912918926421
epoch  7000 dis:  0.7599264363059726 loss:  0.45701966
mejore
best AUC:  0.7599264363059726
epoch  8000 dis:  0.7676438299688787 loss:  0.44468033
mejore
best AUC:  0.7676438299688787
epoch  9000 dis:  0.7844978125646576 loss:  0.44688514
mejore
best AUC:  0.7844978125646576
epoch  10000 dis:  0.7936023980644149 loss:  0.34763336
mejore
best AUC:  0.7936023980644149
epoch

10 min

In [ ]:
losses

[2.0601177,
 1.1278698,
 1.404794,
 1.199521,
 1.2387245,
 4.0489035,
 0.6096073,
 0.45701966,
 0.44468033,
 0.44688514,
 0.34763336,
 0.5445442,
 4.2339935]

In [ ]:
aucs

[0.5012954302627196,
 0.6149333506607484,
 0.657589176761334,
 0.6896867464457391,
 0.7054822704227821,
 0.722411302996846,
 0.742912918926421,
 0.7599264363059726,
 0.7676438299688787,
 0.7844978125646576,
 0.7936023980644149,
 0.8049058094112108,
 0.8162501266976148]

In [ ]:
print(simple_test(sess, discriminator,user_pos_train))
print(simple_test(sess, discriminator,user_pos_test))

0.82422036442449
0.8150057882059594


## minmax

la regularizacion es crucial

In [ ]:
BATCH_SIZE = 1000 #16
import pickle
print ("load model...")


with open( workdir+ "model_dns_IRGANvisual_boots.pkl", 'rb') as f: 
  param = pickle.load(f,encoding="bytes")

generator = GEN(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2,lamda=0.001, param=param, initdelta=0.1,
                    learning_rate=0.01, imageFeatureDim=1536 )

discriminator = DIS(ITEM_NUM, USER_NUM, EMB_DIM,visual_matrix, K2, lamda=0.001, param=None, initdelta=0.1,
                        learning_rate=0.01, imageFeatureDim=1536) #segundo discriminador (el que no es DNS)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())
g1= simple_test(sess, generator,user_pos_train)
d1=simple_test(sess, discriminator,user_pos_train)
print ("gen ", g1)
print ("dis ", d1)

load model...
gen  0.8164278496177005
dis  0.5022067687503954


In [ ]:
simple_test(sess, generator,user_pos_test)

0.8020417637172242

In [ ]:
print(simple_test(sess, generator,user_pos_train))

0.8153143939135364


In [ ]:

def generate_for_d(sess, model, filename):
    data = []
    for u in user_pos_train: #para cada usario
        pos = user_pos_train[u] #sus items 
        rating = sess.run(model.all_rating, {model.u: [all_user.index(u)]}) #completar la matriz 
        rating = np.array(rating[0]) #/ 0.2  # Temperature #infla los rating #plotear que pasa con la sigmoide cuando ponemos temperatura
        exp_rating = np.exp(rating) 
        prob = exp_rating / np.sum(exp_rating)
        #print(u)
        neg = np.random.choice(list(all_items), size=len(pos), p=prob) #muestreo negativo 
        for i in range(len(pos)):
            data.append( str(all_user.index(u)) + '\t' + str(list(all_items).index(pos[i]) ) + '\t' + str(list(all_items).index(neg[i])) ) 

    with open(filename, 'w')as fout:
        fout.write('\n'.join(data))



In [ ]:
# minimax training
dis_log = open('dis_log_boots.txt', 'w')
gen_log = open('gen_log_boots.txt', 'w')
loss_d=[]
auc_epocs_g=[]
auc_epocs_d=[]
loss_g=[]

for epoch in range(15):#15
    print("inicia minmax")
    if epoch >= 0:
        #entrena D 
         for d_epoch in range(1):##100 #para el generador fijo, entrena al discriminador
           # print("empieza a entrenarse el discriminador")
            loss_d_=0
            if d_epoch % 5 == 0:
                generate_for_d(sess, generator, DIS_TRAIN_FILE) #crea un archivo con las tripletas
                #print("pase")
                train_size = file_len(DIS_TRAIN_FILE)
            index = 1
            c=0
            while index < train_size-BATCH_SIZE: #hacer minibatch con etiquetado si es ejemplo positivo o negativo
                if index + BATCH_SIZE <= train_size + 1:
                    input_user, input_item, input_label = get_batch_data(DIS_TRAIN_FILE, index, BATCH_SIZE)
                else:
                    input_user, input_item, input_label = get_batch_data(DIS_TRAIN_FILE, index,
                                                                            train_size - index + 1)
                index += BATCH_SIZE
               # print(index)
                _, loss_dis = sess.run([discriminator.d_updates, discriminator.pre_loss],
                                 feed_dict={discriminator.u: input_user, discriminator.i: input_item, discriminator.label: input_label}) #actualizaciones de los embeddings y bias tales que clasifiquen bien
                loss_d_+=loss_dis
                c+=1
            
            loss_d.append(loss_d_/c)
            auc_actual = simple_test(sess, discriminator, user_pos_train)
            auc_epocs_d.append(auc_actual)
            print ("epoch ", epoch, "dis: ", loss_d_/c)
            if auc_actual > best_AUC:
                print ('mejore: ', auc_actual)
                best_AUC = auc_actual

         for g_epoch in range(1):  #
            #print("empieza a entrenarse el generador")
            c=0
            loss_g_=0
            for u in user_pos_train: #por cada usuario #por cada usuario
                #u= np.random.choice(list(user_pos_train.keys()),1)[0]
                sample_lambda = 0.2 #temperatura
                pos = [list(all_items).index(i) for i in user_pos_train[u]] #peliculas relevantes
                rating = sess.run(generator.all_logits, {generator.u: all_user.index(u)}) #calcula los ratings para ese usuario
                exp_rating = np.exp(rating)
                prob = exp_rating / np.sum(exp_rating)  # prob is generator distribution p_\theta
                pn = (1 - sample_lambda) * prob
                pn[pos] += sample_lambda * 1.0 / len(pos) #pedirle las prob a lo consumido + 0.2/la cantidad de items consumidos
                # Now, pn is the Pn in importance sampling, prob is generator distribution p_\theta
                sample = np.random.choice(np.arange(ITEM_NUM), 2 * len(pos), p=pn) #elige el doble de items c/r a lo consumido
                
                #               # Get reward and adapt it with importance sampling                #######
                reward = sess.run(discriminator.reward, {discriminator.u: all_user.index(u), discriminator.i: sample})
                reward = reward * prob[sample] / pn[sample]
                ######             # Update G              ####################
                _, loss_gen = sess.run([generator.gan_updates, generator.gan_loss],{generator.u: all_user.index(u), generator.i: sample, generator.reward: reward}) #actualiza pesos
                c+=1
                loss_g_+=loss_gen
                
            loss_g.append(loss_g_/c)
            auc_actual = simple_test(sess, generator, user_pos_train)
            auc_epocs_g.append(auc_actual)
            print ("epoch ", epoch, "gen: ", auc_actual)
            buf = '\t'.join([str(auc_actual) ])
            gen_log.write(str(epoch) + '\t' + buf + '\n')
            gen_log.flush()
            if auc_actual > best_AUC:
                print ('mejore: ', auc_actual)
                best_AUC = auc_actual
                generator.save_model(sess, "gan_generator_boots.pkl") #se guarda el mejor modelo, la mejor factorización matricial.

gen_log.close()
dis_log.close()



inicia minmax
epoch  0 dis:  [7.7057953 7.6309066 7.7027345 ... 7.6953998 7.641296  7.771368 ]
epoch  0 gen:  0.8155650781712475
inicia minmax
epoch  1 dis:  [5.669022  5.6958737 5.637288  ... 5.658829  5.748808  5.69489  ]
epoch  1 gen:  0.8138619305945937
inicia minmax
epoch  2 dis:  [7.30206   6.631453  6.5998197 ... 5.242341  5.643454  5.1726227]
epoch  2 gen:  0.8120950229228414
inicia minmax
epoch  3 dis:  [594.89355   83.14791   15.764571 ...  25.180834 113.66809   26.986382]
epoch  3 gen:  0.8099995749257318
inicia minmax
epoch  4 dis:  [8378.723  6530.4175 2474.9802 ... 1941.1824 1125.6984 2615.1045]
epoch  4 gen:  0.8080118800411334
inicia minmax
epoch  5 dis:  [ 14860.909 227441.98   20191.14  ...  33369.184  73123.195  44738.406]
epoch  5 gen:  0.8068472202491901
inicia minmax
epoch  6 dis:  [2438697.  2209621.5 5404261.  ... 1478476.  1629228.  1358188.2]
epoch  6 gen:  0.8043031639295845
inicia minmax
epoch  7 dis:  [2.9203386e+07 7.9842893e+08 3.3795436e+07 ... 4.2170048

20 min

In [ ]:
print(simple_test(sess, discriminator,user_pos_train))
print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, discriminator,user_pos_test))
print(simple_test(sess, generator,user_pos_test))


0.4958307067582737
0.7889865269004773
0.5037783039593355
0.8062087304815215


In [ ]:
with open( "gan_generator_boots.pkl", 'rb') as f: 
  param = pickle.load(f,encoding="bytes")

generator = GEN(ITEM_NUM, USER_NUM, EMB_DIM, visual_matrix, K2,lamda=0.001, param=param, initdelta=0.1,
                    learning_rate=0.01, imageFeatureDim=1536 )
textfile = open(workdir+"IRGAN_visual_boots.txt", "w")
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())


print(simple_test(sess, generator,user_pos_train))
print(simple_test(sess, generator,user_pos_test))

FileNotFoundError: ignored

0.9198166097131708

0.8090884325775983

In [ ]:
auc_epocs_g

[0.7800054441783106,
 0.778972125143461,
 0.7759656284956431,
 0.7743019204271282,
 0.7725090918823623,
 0.7705910719605862,
 0.7680430120701902,
 0.7647846357169482,
 0.7622624390203705,
 0.7607039350521542,
 0.7576851938300642,
 0.7555315762648882,
 0.7535409105374787,
 0.7492700907608366,
 0.7479142114088607]

In [ ]:
[np.mean(i) for i in loss_g]

[0.28027836768560005,
 0.25151376621715094,
 0.23450459214777578,
 0.2208106676480953,
 0.20411755238651422,
 0.19206104918451564,
 0.17876111284815588,
 0.1735719848496098,
 0.16736188621090672,
 0.15523187488698376,
 0.1464639587204439,
 0.13973303543317267,
 0.13236387254039664,
 0.12362026906901184,
 0.12183147811184622]